In [ ]:
!pip install aeon > /dev/null

## Carregando o repositório do GitHub com os dados

In [1]:
!git clone https://github.com/andryll/Cortisol-Estimator-with-DL

Cloning into 'Cortisol-Estimator-with-DL'...


In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPU(s) detected: {gpus}")
    tf.config.experimental.set_memory_growth(gpus[0], True)
else:
    print("No GPU detected. Please ensure your Colab runtime is set to 'GPU' (Runtime -> Change runtime type -> T4 GPU).")

## Carregando e Preparando os Dados


1.   Os dados de treino, teste e validação são carregados e concatenados, de modo a desfazer a divisão feita anteriormente.

2.   Em seguida, os dados de teste são normalizados com o MinMaxScaler



In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler

data_X = np.load("../data/X_train.npy")
data_X = np.concatenate((data_X, np.load("../data/X_val.npy")), axis=0)
data_X = np.concatenate((data_X, np.load("../data/X_test.npy")), axis=0)

data_y = np.load("../data/y_train.npy")
data_y = np.concatenate((data_y, np.load("../data/y_val.npy")), axis=0)
data_y = np.concatenate((data_y, np.load("../data/y_test.npy")), axis=0)

scaler = StandardScaler()
n_samples, n_timesteps, n_features = data_X.shape
data_X_reshaped = data_X.reshape(-1, n_features)

data_X = scaler.fit_transform(data_X_reshaped)
data_X = data_X.reshape(n_samples, n_timesteps, n_features)

# data_X = data_X[:,:,[1,3]]

print("data_X shape:", data_X.shape)
print("data_y shape:", data_y.shape)

data_X shape: (148, 587, 4)
data_y shape: (148,)


## Realizando a Regressão com o FCN

In [3]:
from aeon.regression.deep_learning import FCNRegressor, InceptionTimeRegressor
from aeon.regression.convolution_based import RocketRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping # Importar EarlyStopping

# Realizando o KFold split com 10 folds
n_splits = 10
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

mses_fcn = [] # Lista para armazenar o MSE de cada Fold
r2s_fcn = [] # Lista para armazenar o R2 de cada Fold

mses_it = [] # Lista para armazenar o MSE de cada Fold
r2s_it = [] # Lista para armazenar o R2 de cada Fold

mses_rock = [] # Lista para armazenar o MSE de cada Fold
r2s_rock = [] # Lista para armazenar o R2 de cada Fold

print(f"Realizando o {n_splits}-fold cross-validation...")

# Definir o callback de Early Stopping
early_stopping_callback = EarlyStopping(
    monitor='loss',  # Monitorar a perda de validação
    patience=20,         # Número de épocas sem melhora após o qual o treinamento será interrompido
    restore_best_weights=True # Restaura os pesos do modelo da melhor época
)

# Loop iterando cada uma das pastas
for fold, (train_index, test_index) in enumerate(kf.split(data_X)): # Enumerando as pastas
    print(f"\nFold {fold+1}/{n_splits}")

    # Dividindo os dados de acordo com a pasta atual
    X_train_fold, X_test_fold = data_X[train_index], data_X[test_index]
    y_train_fold, y_test_fold = data_y[train_index], data_y[test_index]

    # Inicializando o RocketRegressor
    rock = RocketRegressor(n_kernels=10000)
    
    # Inicializando o InceptionTimeRegressor com 100 épocas
    it = InceptionTimeRegressor(n_epochs=100,
                                 batch_size=8,
                                 verbose = False,
                                 callbacks=[early_stopping_callback],
                                metrics = ['mse','r2_score'])
    
    # Inicializando o FCNRegressor com 100 épocas
    fcn = FCNRegressor(n_epochs=100,
                       batch_size=8,
                       verbose = False,
                       callbacks=[early_stopping_callback],
                       metrics = ['mse','r2_score'])
    
    # Treinando o Regressor, passando o callback e definindo uma parte para validação
    rock.fit(X_train_fold, y_train_fold)
    it.fit(X_train_fold, y_train_fold)
    fcn.fit(X_train_fold, y_train_fold)

    # Realizando as predições
    y_pred_fold_rock = rock.predict(X_test_fold)
    y_pred_fold_it = it.predict(X_test_fold)
    y_pred_fold_fcn = fcn.predict(X_test_fold)

    # Calculando o MSE para a pasta atual
    mse_fold_rock = mean_squared_error(y_test_fold, y_pred_fold_rock)
    mses_rock.append(mse_fold_rock)
    
    mse_fold_it = mean_squared_error(y_test_fold, y_pred_fold_it)
    mses_it.append(mse_fold_it)
    
    mse_fold_fcn = mean_squared_error(y_test_fold, y_pred_fold_fcn)
    mses_fcn.append(mse_fold_fcn)

    # Calculando o R2 score para a pasta atual
    r2_fold_rock = r2_score(y_test_fold, y_pred_fold_rock)
    r2s_rock.append(r2_fold_rock)
    
    r2_fold_it = r2_score(y_test_fold, y_pred_fold_it)
    r2s_it.append(r2_fold_it)
    
    r2_fold_fcn = r2_score(y_test_fold, y_pred_fold_fcn)
    r2s_fcn.append(r2_fold_fcn)

    print(f"MSE para a pasta {fold+1} (Rocket): {mse_fold_rock:.4f}")
    print(f"R2 Score para a pasta {fold+1} (Rocket): {r2_fold_rock:.4f}")
    
    print(f"MSE para a pasta {fold+1} (InceptionTime): {mse_fold_it:.4f}")
    print(f"R2 Score para a pasta {fold+1} (IncpetionTime): {r2_fold_it:.4f}")
    
    print(f"MSE para a pasta {fold+1} (FCN): {mse_fold_fcn:.4f}")
    print(f"R2 Score para a pasta {fold+1} (FCN): {r2_fold_fcn:.4f}")

# Calculando a média dos MSE e R2
average_mse_rock = np.mean(mses_rock)
average_r2_rock = np.mean(r2s_rock)

average_mse_it = np.mean(mses_it)
average_r2_it = np.mean(r2s_it)

average_mse_fcn = np.mean(mses_fcn)
average_r2_fcn = np.mean(r2s_fcn)

print(f"\nAverage MSE across {n_splits} folds (Rocket): {average_mse_rock:.4f}")
print(f"Average R2 Score across {n_splits} folds (Rocket): {average_r2_rock:.4f}")

print(f"\nAverage MSE across {n_splits} folds (InceptionTime): {average_mse_it:.4f}")
print(f"Average R2 Score across {n_splits} folds (InceptionTime): {average_r2_it:.4f}")

print(f"\nAverage MSE across {n_splits} folds (FCN): {average_mse_fcn:.4f}")
print(f"Average R2 Score across {n_splits} folds (FCN): {average_r2_fcn:.4f}")


Realizando o 10-fold cross-validation...

Fold 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 442ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 457ms/step
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x000001DD0E5BDA80> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 527ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 406ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 

NameError: name 'y_test_fold_rock' is not defined

# Salvar Resultado em Arquivo

In [3]:
import os

# Criando a pasta de resultados
os.makedirs('../results', exist_ok=True)

print(f"Pasta checada/criada com sucesso.")

Pasta checada/criada com sucesso.


In [4]:
import pandas as pd

# Criando a lista de fols + média
fold_labels = [f"Fold {i+1}" for i in range(len(mses_rock))] + ["Average"]

# Criando um dicionário dos resultados
results_data_rock = {
    'Fold': fold_labels,
    'MSE': mses_rock + [average_mse_rock],
    'R2 Score': r2s_rock + [average_r2_rock]
}

results_data_it = {
    'Fold': fold_labels,
    'MSE': mses_rock + [average_mse_rock],
    'R2 Score': r2s_rock + [average_r2_rock]
}

# Criando o DataFrame
results_df_rock = pd.DataFrame(results_data_rock)

# Salvando em CSV
results_df_rock.to_csv("../results/rocket_results.csv", index=False)

print("Resultados Salvos")


Resultados Salvos

Resumo dos Resultados:
       Fold         MSE  R2 Score
0    Fold 1   84.264924 -0.323616
1    Fold 2  198.628871 -0.289132
2    Fold 3  253.076035 -1.267470
3    Fold 4   96.328213 -0.055284
4    Fold 5   52.232235  0.411333
5    Fold 6   15.138998  0.283151
6    Fold 7   91.276321 -0.358587
7    Fold 8  115.295781 -1.739912
8    Fold 9  148.938402 -1.905313
9   Fold 10  167.655400 -0.010645
10  Average  122.283518 -0.525548
